In [1]:
import sys
sys.path.append("../../")
print(sys.path)

from attack import *
FLAGS.input_dir = "../../../official_data/dev_data"
FLAGS.output_dir = "../../../test_data/NonTargetAttackResult"
FLAGS.weight_path = "../../IJCAI19/weight/"
FLAGS.batch_size = 4

from IJCAI19.model.EmbeddedAttackModel import PredictBatch
OfficialModel.WEIGHT_DIR = FLAGS.weight_path

['C:\\ProgramData\\Miniconda3\\python36.zip', 'C:\\ProgramData\\Miniconda3\\DLLs', 'C:\\ProgramData\\Miniconda3\\lib', 'C:\\ProgramData\\Miniconda3', '', 'C:\\ProgramData\\Miniconda3\\lib\\site-packages', 'C:\\ProgramData\\Miniconda3\\lib\\site-packages\\win32', 'C:\\ProgramData\\Miniconda3\\lib\\site-packages\\win32\\lib', 'C:\\ProgramData\\Miniconda3\\lib\\site-packages\\Pythonwin', 'C:\\ProgramData\\Miniconda3\\lib\\site-packages\\IPython\\extensions', 'C:\\Users\\ywilliam\\.ipython', '../../']


Using TensorFlow backend.


In [ ]:
TARGET_ATTACK = True

M = GradSmoothMomentumIterativeMethod
# from cleverhans.attacks import FastGradientMethod
# M = FastGradientMethod
#non targeted with guessed label
attack_params = {"ep_ratio": 0.2, "nb_iter": 100, "target":TARGET_ATTACK}
attack(M, attack_params, targetlabel=TARGET_ATTACK)

{'ep_ratio': 0.2, 'nb_iter': 100, 'target': True, 'eps': 51.0, 'eps_iter': 0.51, 'y_target': <tf.Tensor 'output:0' shape=(?, 110) dtype=float32>, 'clip_min': 0, 'clip_max': 255}


../..\IJCAI19\module\gs_mim.py:204: UserWarning: kwargs is unused and will be removed on or after 2019-04-26.
  warnings.warn("kwargs is unused and will be removed on or after "


INFO:tensorflow:Restoring parameters from ../../IJCAI19/weight/inception_v1/inception_v1.ckpt
INFO:tensorflow:Restoring parameters from ../../IJCAI19/weight/resnet_v1_50/model.ckpt-49800


In [ ]:
#Validation
#copy dev.csv
batch_shape = [FLAGS.batch_size, FLAGS.image_height, FLAGS.image_width, 3]
open(FLAGS.output_dir+'dev.csv', "wb").write(open(FLAGS.input_dir+'dev.csv', "rb").read())
img_loader = ImageLoader(FLAGS.output_dir, batch_shape, targetlabel=TARGET_ATTACK, label_size=FLAGS.num_classes, format='png', label_file='dev.csv')
name = 'vgg_16'
T3 = AttackModel(batch_shape, FLAGS.num_classes, name=name)
Yp, topK, acc = PredictBatch(T3, img_loader)

In [ ]:
#calc
all_shape = (110, batch_shape[1], batch_shape[2], batch_shape[3])
img_loader = ImageLoader(FLAGS.input_dir, all_shape, targetlabel=False, label_size=FLAGS.num_classes, format='png', label_file='dev.csv')
_, X, Y = next(img_loader)
img_loader = ImageLoader(FLAGS.output_dir, all_shape, targetlabel=TARGET_ATTACK, label_size=FLAGS.num_classes, format='png', label_file='dev.csv')
_, Xadv, Yadv = next(img_loader)

print(X.shape, Xadv.shape, Y.shape, Yadv.shape)
print(" mean L2:", calc_l2(X, Xadv))
print("Score and succ:", calc_score(X, Xadv, Yadv.argmax(1), Yp, target=TARGET_ATTACK))

In [ ]:
plot_images(X, Xadv)

In [ ]:
Y.argmax(1)

In [ ]:
Yadv.argmax(1)

In [ ]:
Yp